In [1]:
!pip install gymnasium renderlab

  Obtaining dependency information for gymnasium from https://files.pythonhosted.org/packages/a8/4d/3cbfd81ed84db450dbe73a89afcd8bc405273918415649ac6683356afe92/gymnasium-0.29.1-py3-none-any.whl.metadata
  Obtaining dependency information for renderlab from https://files.pythonhosted.org/packages/76/43/c4f00b499d795417a26171629dabf995bfec61364a1080963678ac1a891c/renderlab-0.1.20230421184216-py3-none-any.whl.metadata
  Obtaining dependency information for farama-notifications>=0.0.1 from https://files.pythonhosted.org/packages/05/2c/ffc08c54c05cdce6fbed2aeebc46348dbe180c6d2c541c7af7ba0aa5f5f8/Farama_Notifications-0.0.4-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/388.3 kB ? eta -:--:--
     ------- ------------------------------- 71.7/388.3 kB 2.0 MB/s eta 0:00:01
     -------------------------------------- 388.3/388.3 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining d

In [2]:
!pip install opencv-python

  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/c7/ec/9dabb6a9abfdebb3c45b0cc52dec901caafef2b2c7e7d6a839ed86d81e91/opencv_python-4.9.0.80-cp37-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/38.6 MB ? eta -:--:--
   ---------------------------------------- 0.1/38.6 MB 1.7 MB/s eta 0:00:24
   - -------------------------------------- 1.6/38.6 MB 20.6 MB/s eta 0:00:02
   ---- ----------------------------------- 4.7/38.6 MB 37.8 MB/s eta 0:00:01
   -------- ------------------------------- 8.0/38.6 MB 46.6 MB/s eta 0:00:01
   ------------ --------------------------- 11.6/38.6 MB 73.1 MB/s eta 0:00:01
   --------------- ------------------------ 15.0/38.6 MB 73.1 MB/s eta 0:00:01
   ------------------- -------------------- 18.8/38.6 MB 73.1 MB/s eta 0:00:01
   ----------------------- ---------------- 22.7/38.6 MB 81.8 MB/s eta 0:00:01
   ------------------------- -------------- 24.3/38.6 MB 65.6 MB/s eta 0:00:01
   

In [3]:
!pip install pygame

  Obtaining dependency information for pygame from https://files.pythonhosted.org/packages/82/61/93ae7afbd931a70510cfdf0a7bb0007540020b8d80bc1d8762ebdc46479b/pygame-2.5.2-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.8 MB 640.0 kB/s eta 0:00:17
   -- ------------------------------------- 0.7/10.8 MB 8.3 MB/s eta 0:00:02
   ------ --------------------------------- 1.7/10.8 MB 13.4 MB/s eta 0:00:01
   ---------- ----------------------------- 2.9/10.8 MB 18.3 MB/s eta 0:00:01
   ---------------- ----------------------- 4.5/10.8 MB 21.9 MB/s eta 0:00:01
   ----------------------- ---------------- 6.2/10.8 MB 24.9 MB/s eta 0:00:01
   ------------------------------ --------- 8.3/10.8 MB 27.9 MB/s eta 0:00:01
   ---------------------------------------  10.7/10.8 MB 36.4 MB/s eta 0:00:01
   ---------------------------------------  10.8/10.8 MB 36.4 MB/s eta 0:00:01
   -----------------

In [4]:
import gymnasium as gym
import random
from IPython.display import clear_output
%config NotebookApp.iopub_msg_rate_limit=10000
import time

In [14]:
#visualise maze:

desc=["SFFF", "FHHH", "FFFF", "HFHF", "FFGF"]

env = gym.make('FrozenLake-v1', desc=desc, map_name="5x5", is_slippery=False, render_mode="human") 

In [15]:
observation, info = env.reset()

In [69]:
# Q-LEARNING

epsilonValue = 0.05
alpha = 0.5

mazeSize = [5,4]
qTable = {}
for i in range(mazeSize[0]*mazeSize[1]):
    qTable[i] = [0,0,0,0]
    
currentState = 0

In [ ]:
def calcPossibleMoves(state):
    possibleMoves = []
    if (state+1) % mazeSize[1] != 0:
        possibleMoves.append(2)
        
    if (state+1) % mazeSize[1] != 1:
        if not state == 0:
            possibleMoves.append(0)
        
    if state > (mazeSize[1]-1):
        if not state == 0:
            possibleMoves.append(3)
    
    if state < ((mazeSize[0] * mazeSize[1]) - mazeSize[1]):
        possibleMoves.append(1)
        
    return possibleMoves

In [ ]:
def nextStep(state):
    x = max(qTable[state])
    if x >= epsilonValue:
        nextMove = qTable[state].index(x)
    else:
        possMoves = calcPossibleMoves(state)
        nextMove = possMoves[random.randrange(len(possMoves))]
    return nextMove

In [ ]:
convergenceThresh = 0.05
converged = False
def updateTable(direction, nextState, reward):
    global currentState
    updated = qTable[currentState][direction] + alpha*(reward + max(qTable[nextState]) - qTable[currentState][direction])
    changeInQ = abs(qTable[currentState][direction] - updated)
    if changeInQ > 0 and changeInQ < convergenceThresh:
        converged = True
    qTable[currentState][direction] = updated
    currentState = nextState

In [80]:
maxEpisodes = 40
currentEpisode = 1

while not converged:
    if currentEpisode == maxEpisodes:
       break
    while currentEpisode < maxEpisodes:
        direction = nextStep(currentState)
        observation, reward, terminated, truncated, info = env.step(direction)

        updateTable(direction, observation, reward)

        if terminated or truncated:
            observation, info = env.reset()
            currentEpisode += 1

        clear_output(wait=True)
        print("Episode: " + str(currentEpisode) + "/" + str(maxEpisodes))
        print("Q-Table:")
        for i in range(len(qTable)):
            print(str(i) + ": " + str(qTable[i]))

error: display Surface quit

In [11]:
env.reset()

(0, {'prob': 1})

In [77]:
env.close()